# Ab Initio Molecular Dynamics using VASP and ASE

Authors: <br>
[Prof. Elvis do A. Soares](https://github.com/elvissoares) <br>
[PhD. Hugo de L. C. Neto](https://github.com/hugo-neto) <br>

Contacts: <br>
[elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/) <br>
[hneto@peq.coppe.ufrj.br](mailto:hneto@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/hugo/Documents/Programs/VASP/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu_mlff'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

In [ ]:
# Importing the VASP calculator of ASE
from ase.calculators.vasp import Vasp

from ase import Atoms
from ase.build import molecule
from ase.io import write, read
from ase.visualize import view

import numpy as np

import os

# AIMD for Si

In [ ]:
# 1) Get the structure
#    - either read your existing POSCAR
atoms   = read('POSCAR')      # or build it with ase.build.bulk, etc.
out_dir = "vasp_run"

In [ ]:
# 2) Attach a VASP calculator with the INCAR-equivalent settings
calc = Vasp(
    directory=out_dir,  # directory where all input files will go
    xc='PBE',
    encut=300,
    gamma=True,
    kpts=(2, 2, 2),        # Monkhorst–Pack grid -> KPOINTS
    ismear=-1,
    sigma=0.0258,
    ediff=1e-6,
    isym=0,
    ivdw=10,
    #Print conf
    lwave=False, lcharg=False,lreal=False,
    #MD
    ibrion=2,   # MD (treat ionic degrees of freedom)
    nsw=10000,  # no of ionic steps
    potim=2,    # MD time step in fs
    #Thermostat
    mdalgo=3,           # Langevin thermostat
    langevin_gamma=1,   # Friction
    langevin_gamma_l=10,# lattice friction
    pmass=10,           # lattice mass
    tebeg=400,          # Temperature
    isif=3,             # "Yes" for everything
    #ML
    ml_lmlff=True,
    ml_mode="train",
    random_seed=[688344966, 0, 0]
)

# 3a) If you ONLY want to generate the input files:
calc.write_input(atoms)
# This writes: INCAR, POSCAR, KPOINTS, POTCAR into vasp_run/

# 3b) If you also want to actually run VASP:
#atoms.calc = calc
#energy = atoms.get_potential_energy()   # writes input files and runs VASP

In [ ]:
# After calc.write_input(atoms) has created vasp_run/INCAR, POSCAR, etc.
iconst_text = """\
LR 1 7
LR 2 7
LR 3 7
LA 2 3 7
LA 1 3 7
LA 1 2 7
LV 7
"""

with open(f"{out_dir}/ICONST", 'w') as f:
    f.write(iconst_text)

Rerunning because it is necessary to manually insert the <b>ICONST</b> file

In [ ]:
calc    = Vasp(restart=True, directory=out_dir)
atoms   = calc.get_atoms()
energy  = atoms.get_potential_energy() # mpirun -np 1 vasp_std